In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date

In [2]:
freq = ['daily', 'weekly', 'monthly']
files = [f'../Data/gspc_{f}_clean.csv' for f in freq]
dfs = [pd.read_csv(file) for file in files]

for df in dfs:
    df['date'] = pd.to_datetime(df['date']).dt.normalize()

daily = dfs[0]
weekly = dfs[1]
monthly = dfs[2]

In [3]:
t = pd.read_csv(f'../Data/tweet_df_stockmarket_only.csv')
avg_sent = t[(t['isRetweet'] == 'f') & (t['isDeleted'] == 'f')]['avg_scores'].mean()

In [4]:
t = pd.read_csv(f'../Data/tweet_df_stockmarket_only.csv')
t.rename(columns={
        'text': 'tweet', 
        'avg_scores': 'avg', 
        'sum_scores': 'sum', 
        'date': 'timestamp',
        'sentiment_category': 'sentiment',
        'word_count': 'words'
        }, inplace=True)
#tweets.columns
columns = [
        'timestamp', 
        'tweet', 
        'avg', 
        'sum', 
        'isRetweet', 
        'isDeleted', 
        'favorites', 
        'retweets',
        'sentiment',
        'words'
        ]

t['isRetweet'] = t['isRetweet'].apply(lambda x: 1 if x == 't' else 0)
t['isDeleted'] = t['isDeleted'].apply(lambda x: 1 if x == 't' else 0)

t['delta_avg'] = t['avg'].apply(lambda x: x - avg_sent)

#t = t[columns][(t['isRetweet'] == 'f') & (t['isDeleted'] == 'f')].reset_index()
t['date'] = pd.to_datetime(t['timestamp']).dt.normalize()
t['month'] = t['date'] + pd.offsets.MonthBegin(-1)
t['year'] = pd.DatetimeIndex(t['timestamp']).year

sentiments = ['positive', 'negative', 'neutral']
for s in sentiments:
        t[s] = np.where(t['sentiment'] == s, 1, 0)

m = t[t['year'] >= 2016]\
        .groupby('month')\
        .agg({
                'tweet': 'count',
                'isRetweet': 'sum',
                'isDeleted': 'sum', 
                'avg': 'mean', 
                'sum': 'sum',
                'delta_avg': 'mean', 
                'retweets': 'sum', 
                'favorites': 'sum', 
                'words': 'sum',
                'positive': 'sum',
                'negative': 'sum',
                'neutral': 'sum'
        })\
        .reset_index()

d = t[t['year'] >= 2016]\
        .groupby('date')\
        .agg({
                'tweet': 'count',
                'isRetweet': 'sum',
                'isDeleted': 'sum', 
                'avg': 'mean', 
                'sum': 'sum',
                'delta_avg': 'mean', 
                'retweets': 'sum', 
                'favorites': 'sum', 
                'words': 'sum',
                'positive': 'sum',
                'negative': 'sum',
                'neutral': 'sum'
        })\
        .reset_index()

mo = pd.merge(m, monthly, how='inner', left_on='month', right_on='date')
mo = mo[['month', 'tweet', 'isRetweet', 'isDeleted', 'avg', 'sum', 'delta_avg', 'retweets', 'favorites', 'words', 'positive', 'negative', 'neutral', '%change', 'up/down']]\
        .reset_index()

da = pd.merge(d, daily, how='inner', on='date')
da = da[['date', 'tweet', 'isRetweet', 'isDeleted', 'avg', 'sum', 'delta_avg', 'retweets', 'favorites', 'words', 'positive', 'negative', 'neutral', '%change', 'up/down']]\
        .reset_index()

In [5]:
mo.rename(columns={
    'tweet': 'tweets',
    'avg': 'avg_sentiment_score',
    'sum': 'sum_sentiment_score',
    'words': 'word_count',
    '%change': 's&p_%change',
    'up/down': 's&p_up/down'
}, inplace=True)

da.rename(columns={
    'tweet': 'tweets',
    'avg': 'avg_sentiment_score',
    'sum': 'sum_sentiment_score',
    'words': 'word_count',
    '%change': 's&p_%change',
    'up/down': 's&p_up/down'
}, inplace=True)

In [6]:
da

,index,date,tweets,isRetweet,isDeleted,avg_sentiment_score,sum_sentiment_score,delta_avg,retweets,favorites,word_count,positive,negative,neutral,s&p_%change,s&p_up/down
0,0,2016-12-08,1,0,0,5.434286,76.08,-0.100675,10063,40673,83,0,0,1,0.002258,up
1,1,2016-12-15,1,0,0,4.963077,64.52,-0.571884,6982,30051,76,0,1,0,0.003665,up
2,2,2017-01-06,1,0,0,5.777143,80.88,0.242182,15421,54861,82,1,0,0,0.002571,up
3,3,2017-01-09,1,0,0,5.107143,71.50,-0.427818,15261,64289,79,0,0,1,-0.002063,down
4,4,2017-01-12,1,0,0,5.060000,45.54,-0.474961,27885,105693,51,0,0,1,-0.000308,down
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,600,2020-10-16,3,1,0,5.392907,279.98,-0.142054,56104,144319,311,1,0,2,-0.002774,down
601,601,2020-10-19,1,0,0,5.824000,145.60,0.289039,12860,52503,144,1,0,0,-0.019103,down
602,602,2020-10-20,1,0,0,5.838571,81.74,0.303611,27778,118997,80,1,0,0,0.001087,up
603,603,2020-10-21,1,0,0,4.936923,64.18,-0.598038,56660,262641,75,0,1,0,-0.001265,down


In [7]:
mo.to_csv('../Data/monthly_summary_v2.csv', index=False)
da.to_csv('../Data/daily_summary_v2.csv', index=False)